In [1]:
import ROOT
import numpy as np
%jsroot on

def adaptive_rebin(h, hu, minweight, minentry):
    edges  = np.array([h.GetBinLowEdge(i) for i in range(1, h.GetNbinsX()+2)])
    counts = np.array([h.GetBinContent(i) for i in range(1, h.GetNbinsX()+1)])
    cum = np.cumsum(counts)
    total = cum[-1]
    min_weight_per_bin = float(minweight)
    min_entries_per_bin = float(minentry)
    print(f"Total weight: {total}, total entries: {hu.GetEntries()}")
    print(f"Minimum weight per bin: {min_weight_per_bin}")
    print(f"Minimum entries per bin: {min_entries_per_bin}")
    new_edges = [edges[0]]
    accum = 0
    accum_entries = 0
    for i, c in enumerate(counts):
        accum += c
        accum_entries += hu.GetBinContent(i+1)
        #print(f"Cycle {i}, accum_entries {accum_entries}")
        if accum >= min_weight_per_bin and i < len(counts) - 1:# and accum_entries >= min_entries_per_bin:
        #if i < len(counts) - 1 and accum_entries >= min_entries_per_bin:
            new_edges.append(edges[i + 1])
            accum = 0
            accum_entries = 0
    #print(f"New edges {new_edges}")
    if new_edges[-1] != edges[-1]:
        new_edges.append(edges[-1])
    print(f"Created {len(new_edges) - 1} bins")
    # Create new histogram
    hnew = ROOT.TH1F(f"{h.GetName()}_adaptivebinning", f"{h.GetTitle()}", len(new_edges) - 1, np.array(new_edges, dtype='float64'))
    hunweight = ROOT.TH1F(f"{h.GetName()}_adaptivebinning_unweighted", f"{h.GetTitle()}", len(new_edges) - 1, np.array(new_edges, dtype='float64'))
    for i in range(1, h.GetNbinsX() + 1):
        x = h.GetBinCenter(i)
        c = h.GetBinContent(i)
        if c != 0:
            hnew.Fill(x, c)

    for i in range(1, hu.GetNbinsX() + 1):
        x = hu.GetBinCenter(i)
        c = hu.GetBinContent(i)
        if c != 0:
            hunweight.Fill(x, c)

    return hnew, hunweight

"""def adaptive_rebin(h, hu, minweight, minentry):
    # Estrai bordi e contenuti dei bin originali
    edges  = np.array([h.GetBinLowEdge(i) for i in range(1, h.GetNbinsX()+2)])
    counts = np.array([h.GetBinContent(i) for i in range(1, h.GetNbinsX()+1)])
    
    # Accumuli per decidere il rebinning
    new_edges = [edges[0]]
    accum_weight = 0
    accum_entries = 0
    for i, c in enumerate(counts):
        accum_weight += c
        accum_entries += hu.GetBinContent(i+1)
        if accum_weight >= minweight or accum_entries >= minentry:
            new_edges.append(edges[i+1])
            accum_weight = 0
            accum_entries = 0
    if new_edges[-1] != edges[-1]:
        new_edges.append(edges[-1])
    
    print(f"Created {len(new_edges)-1} adaptive bins")

    # Crea nuovi istogrammi
    hnew = ROOT.TH1F(f"{h.GetName()}_adaptivebinning", h.GetTitle(),
                     len(new_edges)-1, np.array(new_edges, dtype='float64'))
    hunweight = ROOT.TH1F(f"{h.GetName()}_adaptivebinning_unweighted", h.GetTitle(),
                          len(new_edges)-1, np.array(new_edges, dtype='float64'))

    # Riempie i nuovi bin distribuendo correttamente il contenuto dei vecchi bin
    for j in range(1, hnew.GetNbinsX() + 1):
        low_new = hnew.GetBinLowEdge(j)
        up_new  = hnew.GetBinLowEdge(j+1)
        content = 0
        entries = 0
        for i in range(1, h.GetNbinsX() + 1):
            low_old = h.GetBinLowEdge(i)
            up_old  = h.GetBinLowEdge(i+1)
            # Calcola la frazione di bin vecchio che cade nel nuovo bin
            overlap = max(0, min(up_new, up_old) - max(low_new, low_old))
            if overlap > 0:
                frac = overlap / (up_old - low_old)
                content += h.GetBinContent(i) * frac
                entries += hu.GetBinContent(i) * frac
        hnew.SetBinContent(j, content)
        hunweight.SetBinContent(j, entries)

    return hnew, hunweight"""


"""def variable_binwidth_rebin(h_weighted, h_unweighted, ranges, density=True):

    edges = []
    for Emin, Emax, bw in ranges:
        e = Emin
        while e < Emax:
            edges.append(e)
            e += bw
    edges.append(ranges[-1][1])

    edges = np.array(edges, dtype='float64')

    h_out = ROOT.TH1F(
        f"{h_weighted.GetName()}_varbin",
        h_weighted.GetTitle(),
        len(edges) - 1,
        edges
    )

    for b in range(1, h_out.GetNbinsX() + 1):
        low = h_out.GetBinLowEdge(b)
        up  = h_out.GetBinLowEdge(b+1)

        w_sum = 0.0
        n_sum = 0.0

        for i in range(1, h_weighted.GetNbinsX() + 1):
            x = h_weighted.GetBinCenter(i)
            if low <= x < up:
                w_sum += h_weighted.GetBinContent(i)
                n_sum += h_unweighted.GetBinContent(i)

        if n_sum > 0:
            h_out.SetBinContent(b, w_sum)
            h_out.SetBinError(b, ROOT.TMath.Sqrt(n_sum) * (w_sum / n_sum))
        else:
            h_out.SetBinContent(b, 0)
            h_out.SetBinError(b, 0)

    if density:
        h_out.Scale(1.0, "width")

    return h_out"""

def variable_binwidth_rebin(h_weighted, h_unweighted, ranges,
                            density=True, print_entries=True):
    """
    h_weighted   : TH1 pesato
    h_unweighted : TH1 non pesato (entries)
    ranges       : [(Emin, Emax, binwidth), ...]
    density      : se True fa dN/dE
    print_entries: se True stampa le entries per bin
    """

    # --- costruzione bin edges ---
    edges = []
    for Emin, Emax, bw in ranges:
        e = Emin
        while e < Emax:
            edges.append(e)
            e += bw
    edges.append(ranges[-1][1])
    edges = np.array(edges, dtype='float64')

    # --- istogrammi output ---
    h_out = ROOT.TH1F(
        f"{h_weighted.GetName()}_varbin",
        h_weighted.GetTitle(),
        len(edges) - 1,
        edges
    )

    h_out_unw = ROOT.TH1F(
        f"{h_unweighted.GetName()}_varbin",
        h_unweighted.GetTitle(),
        len(edges) - 1,
        edges
    )

    # --- riempimento ---
    for b in range(1, h_out.GetNbinsX() + 1):
        low = h_out.GetBinLowEdge(b)
        up  = h_out.GetBinLowEdge(b + 1)

        w_sum = 0.0
        n_sum = 0.0

        for i in range(1, h_weighted.GetNbinsX() + 1):
            x = h_weighted.GetBinCenter(i)
            if low <= x < up:
                w_sum += h_weighted.GetBinContent(i)
                n_sum += h_unweighted.GetBinContent(i)

        h_out.SetBinContent(b, w_sum)
        h_out_unw.SetBinContent(b, n_sum)

        if n_sum > 0:
            h_out.SetBinError(b, ROOT.TMath.Sqrt(n_sum) * (w_sum / n_sum))
            h_out_unw.SetBinError(b, ROOT.TMath.Sqrt(n_sum))
        else:
            h_out.SetBinError(b, 0.0)
            h_out_unw.SetBinError(b, 0.0)

    # --- normalizzazione ---
    h_density = None
    if density:
        h_density = h_out.Clone(f"{h_out.GetName()}_density")
        h_density.Scale(1.0, "width")

    # --- stampa entries per bin ---
    if print_entries:
        print("\nBin   Energy range [GeV]        Entries (unweighted)")
        print("---------------------------------------------------")
        for b in range(1, h_out_unw.GetNbinsX() + 1):
            low = h_out_unw.GetBinLowEdge(b)
            up  = h_out_unw.GetBinLowEdge(b + 1)
            n   = int(h_out_unw.GetBinContent(b))
            print(f"{b:3d}   [{low:8.1f}, {up:8.1f})   {n:6d}")

    return h_out, h_out_unw, h_density

def rebin_histogram_ranges(h, ranges, density=False, name_suffix="_rebin"):
    """
    Rebin a TH1 using arbitrary bin ranges.

    Parameters
    ----------
    h : ROOT.TH1
        Input histogram
    ranges : list of (xmin, xmax)
        New bin ranges
    density : bool, optional
        If True, normalize by bin width
    name_suffix : str
        Suffix for output histogram name

    Returns
    -------
    ROOT.TH1
        Rebinned histogram
    """

    # Build bin edges
    edges = [ranges[0][0]]
    for r in ranges:
        edges.append(r[1])

    edges = array('d', edges)

    hnew = ROOT.TH1D(
        h.GetName() + name_suffix,
        h.GetTitle(),
        len(edges) - 1,
        edges
    )

    hnew.Sumw2()

    # Loop over old bins
    for i in range(1, h.GetNbinsX() + 1):
        x = h.GetBinCenter(i)
        c = h.GetBinContent(i)
        e = h.GetBinError(i)

        if c == 0:
            continue

        new_bin = hnew.FindBin(x)
        hnew.AddBinContent(new_bin, c)

        old_err = hnew.GetBinError(new_bin)
        hnew.SetBinError(new_bin, math.sqrt(old_err**2 + e**2))

    if density:
        for i in range(1, hnew.GetNbinsX() + 1):
            width = hnew.GetBinWidth(i)
            hnew.SetBinContent(i, hnew.GetBinContent(i) / width)
            hnew.SetBinError(i, hnew.GetBinError(i) / width)

    return hnew

def generate_txt_file(hnew, hunweight):
    with open("adaptive_histogram.txt", "w") as fout:
        for i in range(1, hnew.GetNbinsX() + 1):
            low  = hnew.GetBinLowEdge(i)
            high = hnew.GetBinLowEdge(i + 1)
            cont = hnew.GetBinContent(i)
            nentries = hunweight.GetBinContent(i)
            fout.write(f"{low:.6g}\t{high:.6g}\t{cont:.6g}\t{nentries:.6g}\n")

Welcome to JupyROOT 6.30/04


In [2]:
fin = ROOT.TFile.Open("/eos/experiment/sndlhc/users/dancc/FLUKA_regenspectrum_18cmB21/NEWREGEN/FLUKA_muons_transpVeto_fullB21.root")
tree = fin.Get("nt")

In [3]:
nbins = 5000000
minweight = 0.013
minentry = 10
h500 = ROOT.TH1D("h500", "", nbins, 0, 5000)
h500_u = ROOT.TH1D("h500_u", "", nbins, 0, 5000)
for i in range(tree.GetEntries()):
    tree.GetEntry(i)
    h500.Fill(tree.E, tree.w)
    h500_u.Fill(tree.E)
#hadapt, hunweight = adaptive_rebin(h500, h500_u, minweight, minentry)
#c10 = ROOT.TCanvas("c10", "c10", 1200, 800)
#c10.Divide(2,1)
#c10.cd(1)
#hadapt.DrawClone("HIST")
#c10.cd(2)
#hunweight.DrawClone("HIST")
#c10.Draw()

In [4]:
c20 = ROOT.TCanvas("c20", "c20", 1600, 1000)
c20.Divide(2,1)
#c20.cd(1)
#tree.Draw("w:E>>h2(500, 0, 5000, 8, 0, 0.08)", "", "COLZ")
c20.cd(1)
#tree.Draw("E>>h(500, 0, 5000)", "w*4*1E5*2.5080308*1.2*9.5", "HIST")
tree.Draw("E>>h(500, 0, 5000)", "w", "HIST")
c20.cd(2)
ranges = [
    (0,    60,  30),
    (60,    120,  60), 
    (120,  250,  65),
    #(250,  400,  75),
    (250,  500,  125),
    (500 , 800 , 150),
    (800 , 2000,  300),   
    (2000, 5000, 500),  
]
"""ranges = [
    (0,    120,  30),
    (120, 190,  60),
    (190,    560,  70),
    (560,  1010,  90),
    (1100,  1400,  100),
    (1400,  2500,  250),
    (2500, 5000, 300)
]"""
h_out, h_unw, h_den = variable_binwidth_rebin(h500, h500_u, ranges, True)

#h_out.Draw("HIST")
print("Integral:", h_out.Integral())
print("Integral den:", h_den.Integral(), "Norm fact:", h_out.Integral()/h_den.Integral())
#h_den.SetBinContent(7, 0.125)
h_den.SetBinContent(7, (0.0023390348069369793/0.1421)*0.115)
h_den.SetBinContent(11, (0.0013723961310461164/0.08381)*0.06)
print('bin content 7', h_den.GetBinContent(11))
h_den.Scale(h_out.Integral()/h_den.Integral())
print('h_den integral', h_den.Integral())
h_den.SetLineColor(ROOT.kRed)
generate_txt_file(h_den, h_unw)
h_den.Draw("HIST SAMES")
c20.Draw()



Bin   Energy range [GeV]        Entries (unweighted)
---------------------------------------------------
  1   [     0.0,     30.0)       38
  2   [    30.0,     60.0)       31
  3   [    60.0,    120.0)       40
  4   [   120.0,    185.0)       45
  5   [   185.0,    250.0)       37
  6   [   250.0,    375.0)       98
  7   [   375.0,    500.0)       95
  8   [   500.0,    650.0)       96
  9   [   650.0,    800.0)       90
 10   [   800.0,   1100.0)      141
 11   [  1100.0,   1400.0)      169
 12   [  1400.0,   1700.0)       95
 13   [  1700.0,   2000.0)       53
 14   [  2000.0,   2500.0)       59
 15   [  2500.0,   3000.0)       40
 16   [  3000.0,   3500.0)       39
 17   [  3500.0,   4000.0)       11
 18   [  4000.0,   4500.0)        8
 19   [  4500.0,   5000.0)        0
Integral: 5.254671995528042
Integral den: 0.08648829438061512 Norm fact: 60.755874921101324
bin content 7 0.0009825052693486214
h_den integral 5.254671904142015


In [5]:
fout = ROOT.TFile.Open("./adaptive_histo.root", "RECREATE")
h_out.Write()
h_unw.Write()
h_den.Write()
fout.Write()
fout.Close()

In [6]:
nbins = 500
fin2 = ROOT.TFile.Open("/eos/experiment/sndlhc/users/dancc/FLUKA_regenspectrum_18cmB21/NEWREGEN/FLUKA_muons_transpVeto_fullB21_total.root")
tree2 = fin2.Get("nt")
fin3 = ROOT.TFile.Open("/eos/experiment/sndlhc/users/dancc/FLUKA_regenspectrum_18cmB21/NOREBIN_REGEN/FLUKA_muons_transpVeto_fullB21_20GeV.root")
tree3 = fin3.Get("nt")
fin4 = ROOT.TFile.Open("/eos/experiment/sndlhc/users/dancc/FLUKA_regenspectrum_18cmB21/NOREBIN_REGEN/FLUKA_muons_transpVeto_fullB21_20_40_80GeV.root")
tree4 = fin4.Get("nt")
h2 = ROOT.TH1F("h2", "Muons original in B21", nbins, 0, 5000)
h3 = ROOT.TH1F("h3", "Muons regen uniform", nbins, 0, 5000)
h3.SetLineColor(ROOT.kRed)
h4 = ROOT.TH1F("h4", "Muons regen gaus 20GeV", nbins, 0, 5000)
h4.SetLineColor(ROOT.kGreen+2)
h5 = ROOT.TH1F("h5", "Muons regen gaus 20-40-8GeV", nbins, 0, 5000)
h5.SetLineColor(ROOT.kMagenta)
c30 = ROOT.TCanvas("c30", "", 1200, 1000)
tree.Draw("E>>h2", "w*4*1E5*2.5080308*1.2*9.5", "HIST")
tree2.Draw("E>>h3", "", "HIST SAMES")
tree3.Draw("E>>h4", "", "HIST SAMES")
tree4.Draw("E>>h5", "", "HIST SAMES")
ROOT.gPad.BuildLegend()
c30.Draw()

In [7]:
c40 = ROOT.TCanvas("c40", "", 1200, 1000)
h5.Draw()
c40.Draw()

In [8]:
fin5 = ROOT.TFile.Open("/eos/experiment/sndlhc/users/dancc/FLUKA_regenspectrum_18cmB21/SWAN_BINNING/FLUKA_muons_transpVeto_fullB21_total.root")
thistree = fin5.Get("nt")
energy_h = ROOT.TH1D("energy_h", "", 5000, 0, 5000)
for i in range(thistree.GetEntries()):
    thistree.GetEntry(i)
    energy_h.Fill(thistree.E)
h_rebinned, h_rebinned_u, h_rebinned_den = variable_binwidth_rebin(energy_h, energy_h, ranges, False)
c100 = ROOT.TCanvas("c100", "c100", 1600, 1000)
c100.Divide(2,1)
c100.cd(1)
energy_h.Draw("HIST")
c100.cd(2)
h_rebinned.Draw("HIST")
c100.Draw()


Bin   Energy range [GeV]        Entries (unweighted)
---------------------------------------------------
  1   [     0.0,     30.0)   21316736
  2   [    30.0,     60.0)   12431182
  3   [    60.0,    120.0)   9929973
  4   [   120.0,    185.0)   2832446
  5   [   185.0,    250.0)   2473086
  6   [   250.0,    375.0)   1983636
  7   [   375.0,    500.0)   3394536
  8   [   500.0,    650.0)   1081640
  9   [   650.0,    800.0)   1449229
 10   [   800.0,   1100.0)   868999
 11   [  1100.0,   1400.0)   843643
 12   [  1400.0,   1700.0)   426769
 13   [  1700.0,   2000.0)   598389
 14   [  2000.0,   2500.0)   181274
 15   [  2500.0,   3000.0)   188594
 16   [  3000.0,   3500.0)    59752
 17   [  3500.0,   4000.0)    20591
 18   [  4000.0,   4500.0)    14133
 19   [  4500.0,   5000.0)     1205


Warning in <TNetXNGFile::Append>: Replacing existing TH1: energy_h_varbin (Potential memory leak).
